### Image Classification using SVM

#### Import necessary libraries

In [1]:
from PIL import Image
import os
import re

import matplotlib.pyplot as plt
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from skimage.feature import hog

C:\anaconda1\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


#### Construct datatset

In [5]:
import zipfile

In [3]:
test_zip_path = 'testing_photos.zip'
train_zip_path = 'training_photos.zip'

In [6]:
with zipfile.ZipFile(train_zip_path, 'r') as train_zip:
    train_zip.extractall()

with zipfile.ZipFile(test_zip_path, 'r') as test_zip:
    test_zip.extractall()

In [17]:
train_folder = 'a1_TrainingPhotos/'
test_folder = 'a2_NewPhotos/'

In [8]:
X_train = []
Y_train = []
X_test = []
Y_test = []

In [13]:
def process_images(folder):
    X = []
    Y = []
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        img = img.resize([32, 64])  # Resize image to a fixed size
        img = img.convert('L')  # Convert image to grayscale
        fd, hog_image = hog(img, orientations=9, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=True)  # Obtain HOG features
        X.append(fd)

        # Obtain labels from file name
        if re.match("Amir*", filename):
            Yt = 0
        elif re.match("Jaya*", filename):
            Yt = 1
        elif re.match("Hir*", filename):
            Yt = 2
        elif re.match("Anupam*", filename):
            Yt = 3
        else:
            Yt = 4

        Y.append(Yt)
    
    return X, Y

In [45]:
X_train, Y_train = process_images(train_folder)

In [46]:
X_test, Y_test = process_images(test_folder)

In [47]:
for i in X_test:
    X_train.append(i)

In [49]:
for i in Y_test:
    Y_train.append(i)

In [51]:
target_names=['Aamir', 'Jaya', 'Hritik', 'Anupam']

In [68]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42)

In [69]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [70]:
x_train.shape

(429, 3780)

In [71]:
y_train

array([1, 0, 2, 0, 2, 0, 1, 2, 2, 3, 3, 1, 0, 1, 2, 0, 0, 0, 3, 1, 0, 1,
       1, 1, 0, 2, 2, 1, 3, 3, 1, 0, 1, 0, 1, 1, 2, 1, 2, 0, 2, 0, 2, 3,
       0, 2, 0, 2, 2, 0, 0, 3, 1, 0, 0, 3, 0, 0, 1, 0, 2, 2, 3, 1, 0, 1,
       1, 0, 1, 2, 3, 0, 0, 0, 0, 1, 0, 1, 1, 0, 2, 2, 0, 1, 0, 2, 2, 0,
       3, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 3, 2, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 3, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1, 1,
       1, 0, 0, 0, 0, 0, 2, 0, 1, 1, 1, 0, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2,
       0, 0, 2, 1, 0, 0, 2, 2, 2, 1, 0, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 2,
       0, 1, 0, 0, 3, 2, 0, 1, 1, 0, 0, 0, 3, 2, 2, 1, 1, 1, 1, 0, 1, 3,
       1, 3, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 0, 0, 2, 2, 2, 2, 0, 1,
       3, 1, 2, 0, 2, 0, 0, 2, 3, 0, 0, 1, 3, 1, 2, 0, 1, 2, 1, 1, 0, 1,
       2, 0, 1, 0, 1, 0, 2, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 3, 0, 0,
       3, 1, 2, 1, 0, 0, 1, 1, 0, 2, 2, 2, 1, 0, 2, 1, 0, 3, 1, 3, 1, 0,
       0, 2, 3, 0, 0, 0, 2, 2, 0, 1, 0, 1, 3, 2, 1,

#### Set parameters and define model

In [72]:
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(x_train, y_train)

print(clf.best_estimator_)

SVC(C=1000.0, class_weight='balanced', gamma=0.005, probability=True)


In [73]:
import joblib
joblib.dump(clf.best_estimator_, 'c2_FR_svm_classifier.pkl', compress = 1)

['c2_FR_svm_classifier.pkl']

#### Test model

In [75]:
y_pred = clf.predict(x_test)
y_pred

array([1, 0, 2, 0, 1, 0, 1, 1, 0, 3, 2, 1, 1, 0, 3, 1, 2, 0, 0, 3, 0, 0,
       0, 2, 2, 0, 2, 0, 0, 1, 2, 0, 3, 2, 1, 1, 0, 0, 2, 0, 0, 2, 1, 1,
       2, 0, 0, 3, 1, 0, 0, 1, 1, 0, 1, 2, 0, 1, 2, 0, 0, 2, 0, 1, 2, 1,
       0, 2, 2, 2, 0, 3, 1, 1, 0, 1, 1, 1, 1, 2, 2, 0, 1, 0, 0, 0, 0, 0,
       2, 0, 1, 0, 1, 2, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 2, 2, 0, 1])

In [77]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.33      0.50      0.40         2
           1       1.00      1.00      1.00         1
           2       0.50      1.00      0.67         1
           3       0.00      0.00      0.00         2

    accuracy                           0.50         6
   macro avg       0.46      0.62      0.52         6
weighted avg       0.36      0.50      0.41         6



C:\anaconda1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda1\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
